<a href="https://colab.research.google.com/github/LC1332/Zero-Haruhi/blob/main/notebook/GLM_Gradio_playground.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- [x] 载入数据
- [x] 生成小说list
- [x] 每本小说生成role list
- [x] 搭建Gradio
- [ ] 生成messages
- [ ] 支持prompt编辑

In [2]:
!pip install -q openai zhipuai
!pip install -q datasets tiktoken

import os
import httpx

os.environ["OPENAI_API_KEY"]  = "set your key here"
# os.environ["OPENAI_API_BASE"] = "如果中转站则使用base"

import openai
from openai import OpenAI
from openai import AsyncOpenAI

from google.colab import userdata

os.environ["OPENAI_API_KEY"]  = userdata.get("OPENAI_API_KEY")
os.environ["OPENAI_API_BASE"] = userdata.get("OPENAI_API_BASE")
os.environ["ZHIPUAI_API_KEY"]  = userdata.get("ZHIPU_API_KEY")

In [3]:
!pip install -q gradio
import gradio as gr

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.9/16.9 MB 61.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 12.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 36.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.0/139.0 kB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 102.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 10.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires kaleido, which is not installed.


In [4]:
from datasets import load_dataset

dataset = load_dataset("silk-road/50-Chinese-Novel-Characters")

Generating train split: 0 examples [00:00, ? examples/s]

In [5]:
novel_list = []

novel2roles = {}

role2datas = {}

from tqdm import tqdm
for data in tqdm(dataset['train']):
    novel = data['book']
    role = data['role']
    if novel not in novel_list:
        novel_list.append(novel)

    if novel not in novel2roles:
        novel2roles[novel] = []

    if role not in novel2roles[novel]:
        novel2roles[novel].append(role)

    role_tuple = (novel, role)

    if role_tuple not in role2datas:
        role2datas[role_tuple] = []

    role2datas[role_tuple].append(data)




100%|██████████| 141985/141985 [00:14<00:00, 9650.09it/s]


In [6]:
%cd /content
!rm -rf /content/Zero-Haruhi
!git clone https://github.com/LC1332/Zero-Haruhi
%cd /content/Zero-Haruhi

/content
Cloning into 'Zero-Haruhi'...
remote: Enumerating objects: 841, done.
remote: Counting objects: 100% (198/198), done.
remote: Compressing objects: 100% (107/107), done.
remote: Total 841 (delta 154), reused 111 (delta 89), pack-reused 643
Receiving objects: 100% (841/841), 3.33 MiB | 10.92 MiB/s, done.
Resolving deltas: 100% (586/586), done.
/content/Zero-Haruhi


In [7]:
from ChatHaruhi.utils import base64_to_float_array

from tqdm import tqdm

for novel in tqdm(novel_list):
    for role in novel2roles[novel]:
        for data in role2datas[(novel, role)]:
            data["vec"] = base64_to_float_array(data["bge_zh_s15"])

100%|██████████| 51/51 [00:26<00:00,  1.90it/s]


In [8]:
def conv2story( role, conversations ):
    lines = [conv["value"] if conv["from"] == "human" else role + ": " + conv["value"] for conv in conversations]
    return "\n".join(lines)

for novel in tqdm(novel_list):
    for role in novel2roles[novel]:
        for data in role2datas[(novel, role)]:
            data["story"] = conv2story( role, data["conversations"] )

100%|██████████| 51/51 [00:00<00:00, 83.13it/s]


In [9]:
!pip install -q peft

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.4/183.4 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 30.2 MB/s eta 0:00:00


In [10]:
from ChatHaruhi import ChatHaruhi
from ChatHaruhi.response_openai import get_response as get_response_openai
from ChatHaruhi.response_zhipu import get_response as get_response_zhipu
from ChatHaruhi.response_GLM_local import get_response as get_response_GLM

get_response = get_response_zhipu

In [11]:
model_choice2func = {
    "openai": get_response_openai,
    "zhipu": get_response_zhipu,
    "GLM": get_response_GLM,
    "default": get_response_GLM
}


In [12]:
narrators = ["叙述者", "旁白","文章作者","作者","Narrator","narrator"]


def package_persona( role_name, world_name ):
    if role_name in narrators:
        return package_persona_for_narrator( role_name, world_name )

    return f"""I want you to act like {role_name} from {world_name}.
If others‘ questions are related with the novel, please try to reuse the original lines from the novel.
I want you to respond and answer like {role_name} using the tone, manner and vocabulary {role_name} would use."""

def package_persona_for_narrator( role_name, world_name ):
    return f"""I want you to act like narrator {role_name} from {world_name}.
当角色行动之后，继续交代和推进新的剧情."""

In [13]:
# target_book = "凡人修仙传"
# target_role = "韩立"

# if target_book not in novel_list:
#     print(f"novel {target_book} not in dataset")

# if target_role not in novel2roles[target_book]:
#     print(f"role {target_role} not in novel {target_book}")

# stories = [data["story"] for data in role2datas[(target_book, target_role)] ]
# vecs = [data["vec"] for data in role2datas[(target_book, target_role)] ]
# persona = package_persona( target_role, target_book )

# persona += "\n{{RAG对话}}\n{{RAG对话}}\n{{RAG对话}}\n"

In [14]:
# chatbot = ChatHaruhi( role_name = target_role, persona = persona , stories = stories, story_vecs= vecs,\
#                      llm = get_response)

# response = chatbot.chat(user = "", text = "鲁鲁道长: 韩道友近来可好？")
# print(response)

In [15]:
# response = chatbot.chat(user = "", text = "鲁鲁道长: *从纳戒中取出一副地图* 听说道友在找金雷竹，在下刚好获得了一些消息")
# print(response)

In [16]:
role_tuple2chatbot = {}


def initialize_chatbot( novel, role ):
    global role_tuple2chatbot
    if (novel, role) not in role_tuple2chatbot:
        persona = package_persona( role, novel )
        persona += "\n{{RAG对话}}\n{{RAG对话}}\n{{RAG对话}}\n"
        stories = [data["story"] for data in role2datas[(novel, role)] ]
        vecs = [data["vec"] for data in role2datas[(novel, role)] ]
        chatbot = ChatHaruhi( role_name = role, persona = persona , stories = stories, story_vecs= vecs,\
                             llm = get_response)
        chatbot.verbose = False

        role_tuple2chatbot[(novel, role)] = chatbot

from tqdm import tqdm
for novel in tqdm(novel_list):
    for role in novel2roles[novel]:
        initialize_chatbot( novel, role )

100%|██████████| 51/51 [00:08<00:00,  5.89it/s]


In [17]:
readme_text = """# 使用说明

选择小说角色

如果你有什么附加信息，添加到附加信息里面就可以

比如"韩立会炫耀自己刚刚学会了Python"

然后就可以开始聊天了

因为这些角色还没有增加Greeting信息，所以之后再开发个随机乱聊功能

# 开发细节

- 采用ChatHaruhi3.0的接口进行prompting
- 这里的数据是用一个7B的tuned qwen模型进行抽取的
- 想看数据可以去看第三个tab
- 抽取模型用了40k左右的GLM蒸馏数据
- 抽取模型是腾讯大哥BPSK训练的

# 总结人物性格

第三个Tab里面，可以显示一个prompt总结人物的性格

复制到openai或者GLM或者Claude进行人物总结


# 这些小说数据从HaruhiZero 0.4模型开始，被加入训练

openai太慢了 今天试试GLM的

不过当前demo是openai的

"""

In [18]:
from ChatHaruhi.response_openai import get_response, async_get_response
import gradio as gr

def get_role_list( novel ):
    new_list = novel2roles[novel]
    new_value = new_list[0]
    return gr.update(choices = new_list, value = new_value)

save_log = "/content/output.txt"

def get_chatbot( novel, role ):
    if (novel, role) not in role_tuple2chatbot:
        initialize_chatbot( novel, role )

    return role_tuple2chatbot[(novel, role)]

import json

def random_chat_callback( novel, role, chat_history):
    datas = role2datas[(novel, role)]

    reesponse_set = set()

    for chat_tuple in chat_history:
        if chat_tuple[1] is not None:
            reesponse_set.add(chat_tuple[1])

    for _ in range(5):
        random_data = random.choice(datas)
        convs = random_data["conversations"]
        n = len(convs)
        index = [x for x in range(0,n,2)]

        for i in index:
            query = convs[i]['value']
            response = convs[i+1]['value']
            if response not in reesponse_set:
                chat_history.append( (query, response) )
                return chat_history

    return chat_history



async def submit_chat( novel, role, user_name, user_text, chat_history, persona_addition_info,model_sel):

    if len(user_text) > 400:
        user_text = user_text[:400]

    if_user_in_text = True

    chatbot = get_chatbot( novel, role )
    chatbot.persona = initialize_persona( novel, role,  persona_addition_info)
    # chatbot.llm_async = async_get_response

    if model_sel in model_choice2func:
        chatbot.llm = model_choice2func[model_sel]
    else:
        chatbot.llm = model_choice2func['default']

    # if model_sel == "openai":
    #     chatbot.llm = get_response_openai
    # else:
    #     chatbot.llm = get_response_zhipu

    history = []

    for chat_tuple in chat_history:
        if chat_tuple[0] is not None:
            history.append( {"speaker":"{{user}}","content":chat_tuple[0]} )
        if chat_tuple[1] is not None:
            history.append( {"speaker":"{{role}}","content":chat_tuple[1]} )

    chatbot.history = history

    input_text = user_text

    if if_user_in_text:
        input_text = user_name + " : " + user_text
        response = chatbot.chat(user = "", text = input_text )
        # response = await chatbot.async_chat(user = "", text = input_text )
    else:
        response = chatbot.chat(user = user_name, text = input_text)
        # response = await chatbot.async_chat(user = user_name, text = input_text)

    chat_history.append( (input_text, response) )

    print_data = {"novel":novel, "role":role, "user_text":input_text, "response":response}

    print(json.dumps(print_data, ensure_ascii=False))

    with open(save_log, "a",encoding = "utf-8") as f:
        f.write(json.dumps(print_data, ensure_ascii=False) + "\n")

    return chat_history


def initialize_persona( novel, role,  persona_addition_info):
    whole_persona = package_persona( role, novel )
    whole_persona += "\n" + persona_addition_info
    whole_persona += "\n{{RAG对话}}\n{{RAG对话}}\n{{RAG对话}}\n"

    return whole_persona

def clean_history( ):
    return []

def clean_input():
    return ""

import random

def generate_summarize_prompt( novel, role_name ):
    whole_prompt = f'''
你在分析小说{novel}中的角色{role_name}
结合小说{novel}中的内容，以及下文中角色{role_name}的对话
判断{role_name}的人物设定、人物特点以及语言风格

{role_name}的对话:
'''
    stories = [data["story"] for data in role2datas[(novel, role_name)] ]

    sample_n = 5

    sample_stories = random.sample(stories, sample_n)

    for story in sample_stories:
        whole_prompt += story + "\n\n"

    return whole_prompt.strip()


with gr.Blocks() as demo:
    gr.Markdown("""# 50本小说的人物测试

    这个interface由李鲁鲁实现，主要是用来看语料的

    增加了随机聊天，支持GLM，openai切换""")

    with gr.Tab("聊天"):
        with gr.Row():
            novel_sel = gr.Dropdown( novel_list, label = "小说", value = "凡人修仙传" , interactive = True)
            role_sel = gr.Dropdown( novel2roles[novel_sel.value], label = "角色", value = "韩立", interactive = True )

        with gr.Row():
            chat_history = gr.Chatbot(height = 600)

        with gr.Row():
            user_name = gr.Textbox(label="user_name", scale = 1, value = "鲁鲁", interactive = True)
            user_text = gr.Textbox(label="user_text", scale = 20)
            submit = gr.Button("submit", scale = 1)

        with gr.Row():
            random_chat = gr.Button("随机聊天", scale = 1)
            clean_message = gr.Button("清空聊天", scale = 1)

        with gr.Row():
            persona_addition_info = gr.TextArea( label = "额外人物设定", value = "",  interactive = True  )

        with gr.Row():
            update_persona = gr.Button("补充人物设定到prompt", scale = 1)
            model_sel_choices = model_choice2func.keys()
            model_sel = gr.Radio(model_sel_choices, interactive = True, scale = 5, value = "Zhipu", label = "模型选择")

        with gr.Row():
            whole_persona = gr.TextArea( label = "完整的system prompt", value = "",  interactive = False  )

        novel_sel.change(fn = get_role_list, inputs = [novel_sel], outputs = [role_sel]).then(fn = initialize_persona, inputs = [novel_sel, role_sel, persona_addition_info], outputs = [whole_persona])

        role_sel.change(fn = initialize_persona, inputs = [novel_sel, role_sel, persona_addition_info], outputs = [whole_persona])

        update_persona.click(fn = initialize_persona, inputs = [novel_sel, role_sel, persona_addition_info], outputs = [whole_persona])

        random_chat.click(fn = random_chat_callback, inputs = [novel_sel, role_sel, chat_history], outputs = [chat_history])

        user_text.submit(fn = submit_chat, inputs = [novel_sel, role_sel, user_name, user_text, chat_history, persona_addition_info,model_sel], outputs = [chat_history]).then(fn = clean_input, inputs = [], outputs = [user_text])
        submit.click(fn = submit_chat, inputs = [novel_sel, role_sel, user_name, user_text, chat_history, persona_addition_info,model_sel], outputs = [chat_history]).then(fn = clean_input, inputs = [], outputs = [user_text])

        clean_message.click(fn = clean_history, inputs = [], outputs = [chat_history])

    with gr.Tab("README"):
        gr.Markdown(readme_text)

    with gr.Tab("辅助人物总结"):
        with gr.Row():
            generate_prompt = gr.Button("生成人物总结prompt", scale = 1)

        with gr.Row():
            whole_prompt = gr.TextArea( label = "复制这个prompt到Openai或者GLM或者Claude进行总结", value = "",  interactive = False  )

        generate_prompt.click(fn = generate_summarize_prompt, inputs = [novel_sel, role_sel], outputs = [whole_prompt])





demo.launch(share=True, debug = True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://007500dea37a362082.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


tokenizer_config.json:   0%|          | 0.00/367 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/110k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/439k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

...

Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

model-00003-of-00003.safetensors:  98%|#########8| 2.64G/2.68G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1473: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


{"novel": "凡人修仙传", "role": "韩立", "user_text": "鲁鲁 : 你好", "response": "*韩立神色如常的走出了人群，往城外走去*"}
{"novel": "凡人修仙传", "role": "韩立", "user_text": "鲁鲁 : 韩道友小心，快闪开!", "response": "*身形一顿，随即嘴角一翘，泛出一丝笑意来*"}
{"novel": "凡人修仙传", "role": "韩立", "user_text": "鲁鲁 : 韩道友手段果然了得，请问刚才那个是什么法器", "response": "*微微一笑* 没什么，不过是件古宝而已。"}
{"novel": "凡人修仙传", "role": "韩立", "user_text": "鲁鲁 : 小心，血鬼门的人又从西边上来了", "response": "*暗叫一声果然如此*\n*身形一晃，人到了一侧的树后*\n*将那鬼头般的怪物，用神识控制住*"}
{"novel": "凡人修仙传", "role": "韩立", "user_text": "鲁鲁 : 话说你知道血鬼门那些邪修，练的都是什么功法吗?", "response": "他们是修鬼道的，应该练的是鬼鬼神神之类的功法吧！"}
{"novel": "凡人修仙传", "role": "韩立", "user_text": "鲁鲁 : 我们在这儿守了三天了，看来他们不回来了。韩道友你后面有什么打算", "response": "后面？没什么打算，我还要去坠魔谷取回自己的宝物。"}
{"novel": "凡人修仙传", "role": "韩立", "user_text": "鲁鲁 : *和韩立一起前往洞府* 你这修炼洞府倒是真大", "response": "*将洞府的门户打开*"}
{"novel": "凡人修仙传", "role": "韩立", "user_text": "鲁鲁 : *指着一堆竹子* 这发光的竹子是啥?", "response": "*笑了笑* 道友不知道吧。这叫磷火，是竹子的一种变异种类，本身会发出微弱的白光，在阴暗潮湿的地方，它们的光芒更微弱，但只要一碰触到外来的热量，就会发出很亮的光，就像被磷石照亮一般。这种竹子不但可以用来做燃料，本身还是一种难得的药材，可入药治病的。"}
{"

{"novel": "凡人修仙传", "role": "韩立", "user_text": "鲁鲁 : 你好", "response": "*韩立神色如常的走出了人群，往城外走去*"}
{"novel": "凡人修仙传", "role": "韩立", "user_text": "鲁鲁 : 韩道友小心，快闪开!", "response": "*身形一顿，随即嘴角一翘，泛出一丝笑意来*"}
{"novel": "凡人修仙传", "role": "韩立", "user_text": "鲁鲁 : 韩道友手段果然了得，请问刚才那个是什么法器", "response": "*微微一笑* 没什么，不过是件古宝而已。"}
{"novel": "凡人修仙传", "role": "韩立", "user_text": "鲁鲁 : 小心，血鬼门的人又从西边上来了", "response": "*暗叫一声果然如此*\n*身形一晃，人到了一侧的树后*\n*将那鬼头般的怪物，用神识控制住*"}
{"novel": "凡人修仙传", "role": "韩立", "user_text": "鲁鲁 : 话说你知道血鬼门那些邪修，练的都是什么功法吗?", "response": "他们是修鬼道的，应该练的是鬼鬼神神之类的功法吧！"}
{"novel": "凡人修仙传", "role": "韩立", "user_text": "鲁鲁 : 我们在这儿守了三天了，看来他们不回来了。韩道友你后面有什么打算", "response": "后面？没什么打算，我还要去坠魔谷取回自己的宝物。"}
{"novel": "凡人修仙传", "role": "韩立", "user_text": "鲁鲁 : *和韩立一起前往洞府* 你这修炼洞府倒是真大", "response": "*将洞府的门户打开*"}
{"novel": "凡人修仙传", "role": "韩立", "user_text": "鲁鲁 : *指着一堆竹子* 这发光的竹子是啥?", "response": "*笑了笑* 道友不知道吧。这叫磷火，是竹子的一种变异种类，本身会发出微弱的白光，在阴暗潮湿的地方，它们的光芒更微弱，但只要一碰触到外来的热量，就会发出很亮的光，就像被磷石照亮一般。这种竹子不但可以用来做燃料，本身还是一种难得的药材，可入药治病的。"}
{"novel": "凡人修仙传", "role": "韩立", "user_text": "鲁鲁 : *指着一堆带着雷电的竹子* 这发光的竹子是啥?", "response": "*脸色微微一变，但马上就恢复如常的说道* 这就是天雷竹，一种稀有的仙家材料。"}

In [ ]:
#备份


def submit_chat( novel, role, user_name, user_text, chat_history, persona_addition_info ):

    if len(user_text) > 400:
        user_text = user_text[:400]

    if_user_in_text = True

    chatbot = get_chatbot( novel, role )
    chatbot.persona = initialize_persona( novel, role,  persona_addition_info)

    history = []

    for chat_tuple in chat_history:
        if chat_tuple[0] is not None:
            history.append( {"speaker":"{{user}}","content":chat_tuple[0]} )
        if chat_tuple[1] is not None:
            history.append( {"speaker":"{{role}}","content":chat_tuple[1]} )

    chatbot.history = history

    input_text = user_text

    if if_user_in_text:
        input_text = user_name + " : " + user_text
        response = chatbot.chat(user = "", text = input_text )
    else:
        response = chatbot.chat(user = user_name, text = input_text)

    chat_history.append( (user_text, response) )

    print_data = {"novel":novel, "role":role, "user_text":input_text, "response":response}

    print(json.dumps(print_data, ensure_ascii=False))

    with open(save_log, "a",encoding = "utf-8") as f:
        f.write(json.dumps(print_data, ensure_ascii=False) + "\n")

    return chat_history